In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Xception

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
import pandas as pd

# Path to dataset directories
train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train"
validation_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Validation"
test_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

# Set batch size and image size
batch_size = 64
img_size = (299, 299)

# Data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training dataset
train_data_x = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the validation dataset
validation_data_x = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the test dataset
test_data_x = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load the pre-trained Xception model without the top layers
base_model = Xception(weights='imagenet', input_shape=(299, 299, 3), include_top=False)

# Freeze the layers of the pre-trained model
base_model.trainable = False

# Add a new output layer for binary classification
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)  # 1 output node with sigmoid activation for binary classification

# Create the fine-tuned model
model = keras.models.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data_x,
    epochs=10,
    validation_data=validation_data_x
)

model.summary()


In [ ]:
# Save the trained model
model.save('xception.h5')

# Save training history
history_df = pd.DataFrame(history.history)
history_df.to_csv('training_history_xception.csv', index=False)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data_x)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%')


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.applications import Xception
from keras.optimizers import Adam
import tensorflow as tf
def learning_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    # Plot training and validation accuracy per epoch
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot training and validation loss per epoch
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

learning_curves(history)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Get true labels and predictions
y_true = test_data_x.classes
y_pred = (model.predict(test_data_x) > 0.5).astype('int32').flatten()

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Class 0', 'Class 1'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for Xception Model')
plt.show()


# EfficientNet V2S

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.optimizers import Adam
import pandas as pd

# Path to dataset directories
train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train"
validation_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Validation"
test_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

# Set batch size and image size
batch_size = 64
img_size = (300, 300)

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training dataset
train_data_v2S = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the validation dataset
validation_data_v2S = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the test dataset
test_data_v2S = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Define the model
model = EfficientNetV2S(weights='imagenet', input_shape=(300, 300, 3), include_top=False)

# Freeze the convolutional base
model.trainable = False

# Add custom classification head
x = keras.layers.GlobalAveragePooling2D()(model.output)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# Compile the model
model = keras.models.Model(inputs=model.input, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data_v2S,
    epochs=10,
    validation_data=validation_data_v2S
)
model.summary()


In [ ]:
# Save the model
model.save('v2s.h5')

# Save training history
history_df = pd.DataFrame(history.history)
history_df.to_csv('training_history_v2s_2.csv', index=False)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data_v2S)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%')


In [ ]:
# Get true labels and predictions for EfficientNetV2-S
y_true_v2s = test_data_v2S.classes
y_pred_v2s = (model.predict(test_data_v2S) > 0.5).astype('int32').flatten()

# Compute confusion matrix for EfficientNetV2-S
cm_v2s = confusion_matrix(y_true_v2s, y_pred_v2s)

# Display confusion matrix for EfficientNetV2-S
disp_v2s = ConfusionMatrixDisplay(confusion_matrix=cm_v2s, display_labels=['Class 0', 'Class 1'])
disp_v2s.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for EfficientNetV2-S Model')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.applications import Xception
from keras.optimizers import Adam
import tensorflow as tf
def learning_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    # Plot training and validation accuracy per epoch
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot training and validation loss per epoch
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

learning_curves(history)


# Efficientnet V2M

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.optimizers import Adam
import pandas as pd

# Path to dataset directories
train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train"
validation_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Validation"
test_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

# Set batch size and image size
batch_size = 64
img_size = (260, 260)

# Data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training dataset
train_data_v2M = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the validation dataset
validation_data_v2M = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the test dataset
test_data_v2M = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Step 1: Load the EfficientNetV2-M model without the top classification layer
base_model = EfficientNetV2M(weights='imagenet', input_shape=(260, 260, 3), include_top=False)

# Step 2: Freeze the convolutional base
base_model.trainable = False

# Step 3: Add custom classification head
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# Step 4: Compile the model
model = keras.models.Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(
    train_data_v2M,
    epochs=10,
    validation_data=validation_data_v2M
)

model.summary()


In [ ]:
# Save the trained model
model.save('v2m.h5')

# Save training history
history_df = pd.DataFrame(history.history)
history_df.to_csv('training_history_v2m_2.csv', index=False)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data_v2M)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.applications import Xception
from keras.optimizers import Adam
import tensorflow as tf
def learning_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    # Plot training and validation accuracy per epoch
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot training and validation loss per epoch
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

learning_curves(history)


In [ ]:
# Get true labels and predictions for EfficientNetV2-M
y_true_v2m = test_data_v2M.classes
y_pred_v2m = (model.predict(test_data_v2M) > 0.5).astype('int32').flatten()

# Compute confusion matrix for EfficientNetV2-M
cm_v2m = confusion_matrix(y_true_v2m, y_pred_v2m)

# Display confusion matrix for EfficientNetV2-M
disp_v2m = ConfusionMatrixDisplay(confusion_matrix=cm_v2m, display_labels=['Class 0', 'Class 1'])
disp_v2m.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for EfficientNetV2-M Model')
plt.show()
